In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s /content/drive/My\ Drive /mydrive
!nvidia-smi

Mounted at /content/drive
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
%%time
%%bash
# INSTALL MINICONDA
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget -q https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
which conda

# INSTALL system packages
apt-get update && apt-get install -qq bc tree sox ffmpeg libsndfile1 libsox-fmt-mp3


# PIP INSTALL NeMo
conda create -n nemo python=3.8 -y
conda init bash
source activate nemo
pip -q install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
cd /mydrive/NeMo/ && pip -q install -r requirements/requirements.txt
cd /mydrive/NeMo/ && pip -q install -e .[asr]
pip -q install wandb torchaudio==0.6.0
pip -q install "util@git+https://git@github.com/dertilo/util.git#egg=util"

source activate nemo
python -c "import torch; print(torch.cuda.is_available())"

# INSTALL ctc-decoder
source activate nemo
apt-get install -qq swig
cd /mydrive/NeMo
git clone https://github.com/NVIDIA/OpenSeq2Seq -b ctc-decoders
mv OpenSeq2Seq/decoders .
rm -rf OpenSeq2Seq
cd decoders
sh ./setup.sh > decoder_setup.log


# EXTRACT DATA
rm -rf /content/corpora
source activate nemo
python
import tarfile
from pathlib import Path
import os

corpora_dir = "/mydrive/data/asr_data/ENGLISH"

#zips = list(Path(corpora_dir).rglob("*processed_mp3.tar.gz"))
zips = [f"{corpora_dir}/tedlium_mp3.tar.gz"]
print(zips)

dest_dir = "/content/corpora/"
os.makedirs(dest_dir,exist_ok=True)
for zipfile in zips:
    with tarfile.open(zipfile, mode="r:gz") as tar:
        tar.extractall(dest_dir)

        

In [3]:
%%time
%%bash

# INFERENCE on MANIFEST
source activate nemo
python -c "import torch; print(torch.cuda.is_available())"

python /mydrive/lfc/run_batch_inference.py --corpora_dir /content/corpora/tedlium_mp3/test --arpa "/mydrive/data/asr_data/ENGLISH/kenlm_vanilla.arpa.gz" --results_dir "/mydrive/data/asr_data/results/test_tedlium_vanilla"

python /mydrive/lfc/run_batch_inference.py --corpora_dir /content/corpora/tedlium_mp3/test --arpa "/mydrive/data/asr_data/ENGLISH/kenlm_tedlium.arpa.gz" --results_dir "/mydrive/data/asr_data/results/test_tedlium_tedlium"


False
Using NGC cloud ASR model QuartzNet5x5LS-En
[NeMo I 2020-11-05 15:21:42 cloud:65] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/QuartzNet5x5LS-En.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet5x5LS-En/cf17856f9aa9ba75ddf7881277884caa/QuartzNet5x5LS-En.nemo
[NeMo I 2020-11-05 15:21:45 common:399] Instantiating model from pre-trained checkpoint
[NeMo I 2020-11-05 15:21:46 features:241] PADDING: 16
[NeMo I 2020-11-05 15:21:46 features:258] STFT using torch
[NeMo I 2020-11-05 15:21:46 modelPT:251] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0b2/QuartzNet5x5LS-En/cf17856f9aa9ba75ddf7881277884caa/QuartzNet5x5LS-En.nemo.
[NeMo I 2020-11-05 15:21:46 collections:173] Dataset loaded with 1144 files totalling 2.64 hours
[NeMo I 2020-11-05 15:21:46 collections:174] 0 files were filtered totalling 0.00 hours
Got WER of 0.2896775102098296
Using NGC cloud ASR model QuartzNet5x5LS-En
[NeMo I 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[NeMo W 2020-11-05 15:21:14 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-05 15:21:14 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-05 15:21:14 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-11-05 15:21:14 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_

CPU times: user 121 ms, sys: 46.4 ms, total: 167 ms
Wall time: 39min 6s


In [3]:
%%time
%%bash
# BUILD KENLM
git clone https://github.com/kpu/kenlm.git
cd kenlm
mkdir -p build
cd build
cmake ..
make -j 4

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 

Cloning into 'kenlm'...


CPU times: user 19.8 ms, sys: 9.23 ms, total: 29 ms
Wall time: 1min 28s


In [11]:
%%time
%%bash
# TRAIN KENLM
## my laptop is faster than colab!
rm -rf kenlm_cache
mkdir kenlm_cache

python3 /mydrive/lfc/kenlm_arpa.py \
    --cache_dir kenlm_cache \
    --input_txt /mydrive/data/asr_data/librispeech-lm-norm.txt.gz \
    --output_dir /mydrive/data/asr_data/kenlm_3_089 \
    --top_k 200000 \
    --kenlm_bins /content/kenlm/build/bin/ \
    --arpa_order 3 --max_arpa_memory "80%" --arpa_prune "0|8|9"




Converting to lowercase and counting word occurrences ...

Saving top 200000 words ...

Creating ARPA file ...

Filtering ARPA file using vocabulary of top-k words ...


40418261it [17:10, 39231.06it/s]=== 1/5 Counting and sorting n-grams ===
Reading /content/kenlm_cache/lower.txt.gz
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 2514862080 bytes == 0x55886256e000 @  0x7efe96ea41e7 0x55885fb6b7b2 0x55885fb0652e 0x55885fae52fb 0x55885fad1076 0x7efe95a21b97 0x55885fad2bba
tcmalloc: large alloc 8382865408 bytes == 0x5588f83ca000 @  0x7efe96ea41e7 0x55885fb6b7b2 0x55885fb5a7da 0x55885fb5b218 0x55885fae5318 0x55885fad1076 0x7efe95a21b97 0x55885fad2bba
****************************************************************************************************
Unigram tokens 803288729 types 973676
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:11684112 2:3795199488 3:7115999232
tcmalloc: large alloc 7116005376 bytes == 0x558862460000 @  0x7efe96ea41e7 0x55885fb6b7b2 0x55885fb5a7da 0x55885fb5b218 0x55885fae58e7 0x55885fad1076 0x7efe95a21b97 0x55885fad2bba
tcmalloc: large al

CPU times: user 1.12 s, sys: 222 ms, total: 1.34 s
Wall time: 34min 1s


In [ ]:
%%time
%%bash
# INFERENCE single FILE
source activate nemo
python

import nemo.collections.asr as nemo_asr
import numpy as np

asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="QuartzNet5x5LS-En")

# softmax implementation in NumPy
def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])


# let's do inference once again but without decoder
data_path = "/content/dev-other_processed_wav"
files = [f"{data_path}/content_SPANISH_raw_dev-other_LibriSpeech_dev-other_116_288045_116-288045-0000.wav"]

transcript = asr_model.transcribe(paths2audio_files=files)[0]
print(f'Transcript: "{transcript}"')

logits = asr_model.transcribe(files, logprobs=True)[0].cpu().numpy()
probs = softmax(logits)
print(probs)